In [1]:
import snowflake.connector
import os

# Conexión Snowflake (ya tienes estas env vars)
conn = snowflake.connector.connect(
    account=os.getenv("SF_ACCOUNT"),
    user=os.getenv("SF_USER"),
    password=os.getenv("SF_PASSWORD"),
    warehouse=os.getenv("SF_WAREHOUSE"),
    database=os.getenv
    ("SF_DATABASE"),
    schema="PUBLIC",     # luego cambia a ANALYTICS
    role=os.getenv("SF_ROLE"),
)
cur = conn.cursor()

# 1️⃣ Crear esquema ANALYTICS si no existe
cur.execute("""
CREATE SCHEMA IF NOT EXISTS ANALYTICS;
""")

# 2️⃣ Crear OBT (One Big Table) como TABLE física
cur.execute("""
CREATE OR REPLACE TABLE ANALYTICS.OBT_TRIPS AS
SELECT
    -- 🕒 TIME DIMENSIONS
    pickup_datetime,
    dropoff_datetime,
    TO_DATE(pickup_datetime)               AS pickup_date,
    TO_CHAR(pickup_datetime, 'HH24')       AS pickup_hour,
    TO_CHAR(dropoff_datetime, 'HH24')      AS dropoff_hour,
    TO_CHAR(pickup_datetime, 'DY')         AS day_of_week,
    EXTRACT(MONTH FROM pickup_datetime)    AS month,
    EXTRACT(YEAR  FROM pickup_datetime)    AS year,

    -- 📍 LOCATIONS (from enriched table)
    PULocationID         AS pu_location_id,
    PICKUP_ZONE          AS pu_zone,
    PICKUP_BOROUGH       AS pu_borough,
    DOLocationID         AS do_location_id,
    DROPOFF_ZONE         AS do_zone,
    DROPOFF_BOROUGH      AS do_borough,

    -- 🚕 SERVICE & CODES
    SERVICE_TYPE,
    VENDORID             AS vendor_id,
    VENDOR_DESC          AS vendor_name,
    RATECODEID           AS rate_code_id,
    RATE_CODE_DESC,
    PAYMENT_TYPE,
    PAYMENT_TYPE_DESC,
    TRIP_TYPE,

    -- 🧳 TRIP INFO
    PASSENGER_COUNT,
    TRIP_DISTANCE,
    STORE_AND_FWD_FLAG,

    -- 💵 FARE DETAIL
    FARE_AMOUNT,
    EXTRA,
    MTA_TAX,
    TIP_AMOUNT,
    TOLLS_AMOUNT,
    IMPROVEMENT_SURCHARGE,
    CONGESTION_SURCHARGE,
    AIRPORT_FEE,
    TOTAL_AMOUNT,

    -- 🧮 DERIVED METRICS
    DATEDIFF('minute', pickup_datetime, dropoff_datetime) AS trip_duration_min,
    CASE
        WHEN DATEDIFF('minute', pickup_datetime, dropoff_datetime) > 0
        THEN (TRIP_DISTANCE / (DATEDIFF('minute', pickup_datetime, dropoff_datetime) / 60.0))
        ELSE NULL
    END AS avg_speed_mph,
    CASE
        WHEN FARE_AMOUNT > 0 THEN (TIP_AMOUNT / FARE_AMOUNT) * 100
        ELSE 0
    END AS tip_pct,

    -- 🗃 LINEAGE / METADATA
    RUN_ID,
    INGESTED_AT_UTC,
    SOURCE_YEAR,
    SOURCE_MONTH
FROM SILVER.NYC_ALL_TAXIS_ENRICHED;
""")

# 3️⃣ Confirmar filas en la OBT
cur.execute("SELECT COUNT(*) FROM ANALYTICS.OBT_TRIPS;")
print("✅ OBT_TRIPS generada. Total filas:", cur.fetchone()[0])

cur.close()
conn.close()


✅ OBT_TRIPS generada. Total filas: 770119593


In [3]:
import snowflake.connector
import os

# 🔁 REUTILIZA LAS PLANTILLAS (necesarias para inserciones)
YELLOW_INSERT_TMPL = """
INSERT INTO SILVER.NYC_ALL_TAXITRIPS
SELECT *
FROM (
    SELECT
        MD5(
            COALESCE(TO_CHAR(tpep_pickup_datetime, 'YYYY-MM-DD HH24:MI:SS.FF3'),'') || '|' ||
            COALESCE(TO_CHAR(tpep_dropoff_datetime,'YYYY-MM-DD HH24:MI:SS.FF3'),'') || '|' ||
            COALESCE(TO_VARCHAR(VendorID),'') || '|' ||
            COALESCE(TO_VARCHAR(PULocationID),'') || '|' ||
            COALESCE(TO_VARCHAR(DOLocationID),'') || '|' ||
            COALESCE(TO_VARCHAR(total_amount),'') || '|' ||
            'YELLOW'
        ) AS HASH_KEY,
        tpep_pickup_datetime::TIMESTAMP_NTZ  AS PICKUP_DATETIME,
        tpep_dropoff_datetime::TIMESTAMP_NTZ AS DROPOFF_DATETIME,
        VendorID                              AS VENDORID,
        CASE VendorID
            WHEN 1 THEN 'Creative Mobile Technologies (CMT)'
            WHEN 2 THEN 'VeriFone Transportation Systems (VTS)'
            WHEN 4 THEN 'DDS'
            WHEN 5 THEN 'VIP'
            ELSE 'Unknown'
        END AS VENDOR_DESC,
        RatecodeID                            AS RATECODEID,
        CASE RatecodeID
            WHEN 1 THEN 'Standard rate'
            WHEN 2 THEN 'JFK'
            WHEN 3 THEN 'Newark'
            WHEN 4 THEN 'Nassau/Westchester'
            WHEN 5 THEN 'Negotiated fare'
            WHEN 6 THEN 'Group ride'
            ELSE 'Unknown'
        END AS RATE_CODE_DESC,
        passenger_count                       AS PASSENGER_COUNT,
        trip_distance                         AS TRIP_DISTANCE,
        PULocationID                          AS PULOCATIONID,
        DOLocationID                          AS DOLOCATIONID,
        store_and_fwd_flag                    AS STORE_AND_FWD_FLAG,
        payment_type                          AS PAYMENT_TYPE,
        CASE payment_type
            WHEN 1 THEN 'Credit card'
            WHEN 2 THEN 'Cash'
            WHEN 3 THEN 'No charge'
            WHEN 4 THEN 'Dispute'
            WHEN 5 THEN 'Unknown'
            WHEN 6 THEN 'Voided trip'
            ELSE 'Unknown'
        END AS PAYMENT_TYPE_DESC,
        fare_amount                           AS FARE_AMOUNT,
        extra                                 AS EXTRA,
        mta_tax                               AS MTA_TAX,
        tip_amount                            AS TIP_AMOUNT,
        tolls_amount                          AS TOLLS_AMOUNT,
        improvement_surcharge                 AS IMPROVEMENT_SURCHARGE,
        total_amount                          AS TOTAL_AMOUNT,
        congestion_surcharge                  AS CONGESTION_SURCHARGE,
        cbd_congestion_fee                    AS CBD_CONGESTION_FEE,
        airport_fee                           AS AIRPORT_FEE,
        CAST(NULL AS FLOAT)                   AS EHAIL_FEE,
        CAST(NULL AS FLOAT)                   AS TRIP_TYPE,
        'YELLOW'                              AS SERVICE_TYPE,
        SOURCE_YEAR, SOURCE_MONTH, RUN_ID, INGESTED_AT_UTC, SOURCE_PATH
    FROM RAW.NYC_YELLOW_TAXIS
    WHERE SOURCE_YEAR = {yy} AND SOURCE_MONTH = {mm}
) S
WHERE NOT EXISTS (
    SELECT 1 FROM SILVER.NYC_ALL_TAXITRIPS T WHERE T.HASH_KEY = S.HASH_KEY
);
"""

GREEN_INSERT_TMPL = """
INSERT INTO SILVER.NYC_ALL_TAXITRIPS
SELECT *
FROM (
    SELECT
        MD5(
            COALESCE(TO_CHAR(lpep_pickup_datetime, 'YYYY-MM-DD HH24:MI:SS.FF3'),'') || '|' ||
            COALESCE(TO_CHAR(lpep_dropoff_datetime,'YYYY-MM-DD HH24:MI:SS.FF3'),'') || '|' ||
            COALESCE(TO_VARCHAR(VendorID),'') || '|' ||
            COALESCE(TO_VARCHAR(PULocationID),'') || '|' ||
            COALESCE(TO_VARCHAR(DOLocationID),'') || '|' ||
            COALESCE(TO_VARCHAR(total_amount),'') || '|' ||
            'GREEN'
        ) AS HASH_KEY,
        lpep_pickup_datetime::TIMESTAMP_NTZ   AS PICKUP_DATETIME,
        lpep_dropoff_datetime::TIMESTAMP_NTZ  AS DROPOFF_DATETIME,
        VendorID                              AS VENDORID,
        CASE VendorID
            WHEN 1 THEN 'Creative Mobile Technologies (CMT)'
            WHEN 2 THEN 'VeriFone Transportation Systems (VTS)'
            WHEN 4 THEN 'DDS'
            WHEN 5 THEN 'VIP'
            ELSE 'Unknown'
        END AS VENDOR_DESC,
        RatecodeID                            AS RATECODEID,
        CASE RatecodeID
            WHEN 1 THEN 'Standard rate'
            WHEN 2 THEN 'JFK'
            WHEN 3 THEN 'Newark'
            WHEN 4 THEN 'Nassau/Westchester'
            WHEN 5 THEN 'Negotiated fare'
            WHEN 6 THEN 'Group ride'
            ELSE 'Unknown'
        END AS RATE_CODE_DESC,
        passenger_count                       AS PASSENGER_COUNT,
        trip_distance                         AS TRIP_DISTANCE,
        PULocationID                          AS PULOCATIONID,
        DOLocationID                          AS DOLOCATIONID,
        store_and_fwd_flag                    AS STORE_AND_FWD_FLAG,
        payment_type                          AS PAYMENT_TYPE,
        CASE payment_type
            WHEN 1 THEN 'Credit card'
            WHEN 2 THEN 'Cash'
            WHEN 3 THEN 'No charge'
            WHEN 4 THEN 'Dispute'
            WHEN 5 THEN 'Unknown'
            WHEN 6 THEN 'Voided trip'
            ELSE 'Unknown'
        END AS PAYMENT_TYPE_DESC,
        fare_amount                           AS FARE_AMOUNT,
        extra                                 AS EXTRA,
        mta_tax                               AS MTA_TAX,
        tip_amount                            AS TIP_AMOUNT,
        tolls_amount                          AS TOLLS_AMOUNT,
        improvement_surcharge                 AS IMPROVEMENT_SURCHARGE,
        total_amount                          AS TOTAL_AMOUNT,
        congestion_surcharge                  AS CONGESTION_SURCHARGE,
        cbd_congestion_fee                    AS CBD_CONGESTION_FEE,
        CAST(NULL AS FLOAT)                   AS AIRPORT_FEE,
        ehail_fee                             AS EHAIL_FEE,
        trip_type                             AS TRIP_TYPE,
        'GREEN'                               AS SERVICE_TYPE,
        SOURCE_YEAR, SOURCE_MONTH, RUN_ID, INGESTED_AT_UTC, SOURCE_PATH
    FROM RAW.NYC_GREEN_TAXIS
    WHERE SOURCE_YEAR = {yy} AND SOURCE_MONTH = {mm}
) S
WHERE NOT EXISTS (
    SELECT 1 FROM SILVER.NYC_ALL_TAXITRIPS T WHERE T.HASH_KEY = S.HASH_KEY
);
"""

# == Desde aquí va el código del test de idempotencia ==


In [5]:
import snowflake.connector
import os

# Conexión
conn = snowflake.connector.connect(
    account=os.getenv("SF_ACCOUNT"),
    user=os.getenv("SF_USER"),
    password=os.getenv("SF_PASSWORD"),
    warehouse=os.getenv("SF_WAREHOUSE"),
    database=os.getenv("SF_DATABASE"),
    schema="PUBLIC",
    role=os.getenv("SF_ROLE"),
)
cur = conn.cursor()

# Mes de prueba
TEST_YEAR = 2017
TEST_MONTH = 10

print(f"\n🔍 Probando idempotencia para {TEST_YEAR}-{TEST_MONTH:02d}")

# 1️⃣ Contar filas antes del insert
cur.execute(f"""
SELECT COUNT(*) FROM SILVER.NYC_ALL_TAXITRIPS
WHERE SOURCE_YEAR = {TEST_YEAR} AND SOURCE_MONTH = {TEST_MONTH};
""")
initial_count = cur.fetchone()[0]
print(f"📊 Filas iniciales (antes de carga): {initial_count}")

# 2️⃣ Ejecutar inserción YELLOW
print("🟡 Insertando YELLOW de prueba...")
cur.execute(YELLOW_INSERT_TMPL.format(yy=TEST_YEAR, mm=TEST_MONTH))
print("✅ YELLOW insert OK")

# 3️⃣ Ejecutar inserción GREEN
print("🟢 Insertando GREEN de prueba...")
cur.execute(GREEN_INSERT_TMPL.format(yy=TEST_YEAR, mm=TEST_MONTH))
print("✅ GREEN insert OK")

# 4️⃣ Contar filas después de primer insert
cur.execute(f"""
SELECT COUNT(*) FROM SILVER.NYC_ALL_TAXITRIPS
WHERE SOURCE_YEAR = {TEST_YEAR} AND SOURCE_MONTH = {TEST_MONTH};
""")
after_first = cur.fetchone()[0]
print(f"📊 Filas después del primer insert: {after_first}")

# 5️⃣ Reintentar carga del mismo mes (2º insert)
print("🔁 Reintentando carga del mismo mes (idempotencia test)...")
cur.execute(YELLOW_INSERT_TMPL.format(yy=TEST_YEAR, mm=TEST_MONTH))
cur.execute(GREEN_INSERT_TMPL.format(yy=TEST_YEAR, mm=TEST_MONTH))

# 6️⃣ Contar filas después del segundo insert
cur.execute(f"""
SELECT COUNT(*) FROM SILVER.NYC_ALL_TAXITRIPS
WHERE SOURCE_YEAR = {TEST_YEAR} AND SOURCE_MONTH = {TEST_MONTH};
""")
after_second = cur.fetchone()[0]

print(f"📊 Filas después del segundo insert: {after_second}")

# ✅ Test Final
if after_first == after_second:
    print("\n🟢 ✅ IDOMPOTENCIA COMPROBADA – No se duplicaron filas.")
else:
    print("\n🔴 ❌ ERROR – Filas duplicadas. Revisar HASH_KEY / INSERT.")

cur.close()
conn.close()



🔍 Probando idempotencia para 2017-10
📊 Filas iniciales (antes de carga): 10694409
🟡 Insertando YELLOW de prueba...
✅ YELLOW insert OK
🟢 Insertando GREEN de prueba...
✅ GREEN insert OK
📊 Filas después del primer insert: 10694409
🔁 Reintentando carga del mismo mes (idempotencia test)...
📊 Filas después del segundo insert: 10694409

🟢 ✅ IDOMPOTENCIA COMPROBADA – No se duplicaron filas.
